In [174]:
import torch
import csv
import os
from transformers import ElectraTokenizer, ElectraConfig, AutoModelForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
import logging
from fastprogress.fastprogress import master_bar, progress_bar
from transformers.modeling_electra import ElectraModel, ElectraPreTrainedModel
import glob
logger = logging.getLogger(__name__)
import tqdm
import numpy as np

In [2]:
model = torch.load("./model/koELECTRA_epoch7")

In [4]:
input_file = './data/nsmc/ko_data.txt'
result_file = './koELECTRA_data.csv'

In [9]:
def load_data(path):
    result = []
    with open(path, "r") as f:
        for line in f:
            result.append(line.strip())
    return result

In [10]:
expectations = ['Predicted']

print(load_data(input_file)[:3])

['정말 많이 울었던 영화입니다.', '시간 낭비예요.', '포스터를 저렇게밖에 만들지 못했던 제작자의 소심함에 침을 뱉고 싶다.']


In [31]:
args = {
  "task": "nsmc",
  "data_dir": "data",
  "ckpt_dir": "ckpt",
  "train_file": "ratings_train.txt",
  "dev_file": "",
  "test_file": "ko_data.txt",
  "evaluate_test_during_training": True,
  "eval_all_checkpoints": True,
  "save_optimizer": False,
  "do_lower_case": False,
  "do_train": True,
  "do_eval": True,
  "max_seq_len": 50,
  "num_train_epochs": 8,
  "weight_decay": 0.0,
  "gradient_accumulation_steps": 1,
  "adam_epsilon": 1e-8,
  "warmup_steps": 0,
  "max_steps": -1,
  "max_grad_norm": 1.0,
  "no_cuda": False,
  "model_type": "koelectra-base-v2",
  "model_name_or_path": "monologg/koelectra-base-v2-discriminator",
  "output_dir": "koelectra-base-v2-nsmc-ckpt",
  "seed": 42,
  "train_batch_size": 64,
  "eval_batch_size": 128,
  "logging_steps": 2000,
  "save_steps": 2000,
  "learning_rate": 5e-5
}

In [83]:
class NsmcProcessor(object):
    """Processor for the NSMC data set """

    def __init__(self, args):
        self.args = args

    def get_labels(self):
        return ["0", "1"]

    @classmethod
    def _read_file(cls, input_file):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8") as f:
            lines = []
            for line in f:
                lines.append(line.strip())
            return lines

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line
            if i % 10000 == 0:
                logger.info(line)
            examples.append(InputExample(guid=guid, text_a=text_a, text_b=None))
        return examples

    def get_examples(self, mode):
        """
        Args:
            mode: train, dev, test
        """
        file_to_read = None
        if mode == 'train':
            file_to_read = self.args.train_file
        elif mode == 'dev':
            file_to_read = self.args.dev_file
        elif mode == 'test':
            file_to_read = "ko_data.txt"
        logger.info("LOOKING AT {}".format(os.path.join("data", "nsmc", file_to_read)))
        return self._create_examples(self._read_file(os.path.join("data", "nsmc", file_to_read)), mode)

In [84]:
class InputExample(object):
    """
    A single training/test example for simple sequence classification.
    """

    def __init__(self, guid, text_a, text_b):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [91]:
class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, attention_mask, token_type_ids):
        self.input_ids = input_ids
        self.attention_mask = attention_mask
        self.token_type_ids = token_type_ids

    def __repr__(self):
        return str(self.to_json_string())

    def to_dict(self):
        """Serializes this instance to a Python dictionary."""
        output = copy.deepcopy(self.__dict__)
        return output

    def to_json_string(self):
        """Serializes this instance to a JSON string."""
        return json.dumps(self.to_dict(), indent=2, sort_keys=True) + "\n"

In [98]:
def seq_cls_load_and_cache_examples(args, tokenizer, mode):
    processor = NsmcProcessor(args)
    output_mode = "classification"
    # Load data features from cache or dataset file
    cached_features_file = os.path.join('./data', "cached_nsmc_koelectra-base-v2-discriminator_50_{}".format(mode))
    if os.path.exists(cached_features_file):
        logger.info("Loading features from cached file %s", cached_features_file)
        features = torch.load(cached_features_file)
    else:
        logger.info("Creating features from dataset file at %s", "./data")
        if mode == "train":
            examples = processor.get_examples("train")
        elif mode == "dev":
            examples = processor.get_examples("dev")
        elif mode == "test":
            examples = processor.get_examples("test")
        else:
            raise ValueError("For mode, only train, dev, test is avaiable")
        features = seq_cls_convert_examples_to_features(args, examples, tokenizer, 50, "nsmc")
        logger.info("Saving features into cached file %s", cached_features_file)
        torch.save(features, cached_features_file)

    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_attention_mask = torch.tensor([f.attention_mask for f in features], dtype=torch.long)
    all_token_type_ids = torch.tensor([f.token_type_ids for f in features], dtype=torch.long)
#     if output_mode == "classification":
#         all_labels = torch.tensor([f.label for f in features], dtype=torch.long)
#     elif output_mode == "regression":
#         all_labels = torch.tensor([f.label for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids)
    return dataset

In [99]:
def seq_cls_convert_examples_to_features(
        args,
        examples,
        tokenizer,
        max_length,
        task
):
    processor = NsmcProcessor(args)
    label_list = processor.get_labels()
    logger.info("Using label list {} for task {}".format(label_list, "nsmc"))
    output_mode = "classification"
    logger.info("Using output mode {} for task {}".format(output_mode, "nsmc"))

    label_map = {label: i for i, label in enumerate(label_list)}

    def label_from_example(example):
        if output_mode == "classification":
            return label_map[example.label]
        elif output_mode == "regression":
            return float(example.label)
        raise KeyError(output_mode)

    batch_encoding = tokenizer.batch_encode_plus(
        [(example.text_a, example.text_b) for example in examples], max_length=max_length, pad_to_max_length=True
    )

    features = []
    for i in range(len(examples)):
        inputs = {k: batch_encoding[k][i] for k in batch_encoding}
        if "token_type_ids" not in inputs:
            inputs["token_type_ids"] = [0] * len(inputs["input_ids"])  # For xlm-roberta

        feature = InputFeatures(**inputs)
        features.append(feature)

    for i, example in enumerate(examples[:5]):
        logger.info("*** Example ***")
        logger.info("guid: {}".format(example.guid))
        logger.info("input_ids: {}".format(" ".join([str(x) for x in features[i].input_ids])))
        logger.info("attention_mask: {}".format(" ".join([str(x) for x in features[i].attention_mask])))
        logger.info("token_type_ids: {}".format(" ".join([str(x) for x in features[i].token_type_ids])))

    return features

In [100]:
tokenizer = ElectraTokenizer.from_pretrained("monologg/koelectra-base-v2-discriminator", do_lower_case=False)

In [101]:
data_set = seq_cls_load_and_cache_examples(args, tokenizer, mode="test")

In [116]:
dataloader = DataLoader(data_set, batch_size= 128)

In [152]:
len(dataloader)

88

In [125]:
class ElectraForSequenceClassification(ElectraPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.electra = ElectraModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

        self.init_weights()

    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            token_type_ids=None,
            position_ids=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
    ):
        discriminator_hidden_states = self.electra(
            input_ids, attention_mask, token_type_ids, position_ids, head_mask, inputs_embeds
        )
        pooled_output = discriminator_hidden_states[0][:, 0]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + discriminator_hidden_states[1:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

In [145]:
checkpoints = list(
            os.path.dirname(c) for c in sorted(glob.glob("./ckpt/koelectra-base-v2-nsmc-ckpt" + "/**/" + "pytorch_model.bin", recursive=True)))

In [170]:
checkpoints

['./ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-10000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-12000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-14000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-16000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-18000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-2000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-20000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-4000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-6000',
 './ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-8000']

In [150]:
global_step = checkpoints[0].split("-")[-1]

In [195]:
model = ElectraForSequenceClassification.from_pretrained('./ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-2000')


NameError: name 'nn' is not defined

In [196]:
for batch in progress_bar(dataloader):
        model.eval()
        batch = tuple(t.to(args.device) for t in batch)

        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "labels": batch[3]
            }
            if args.model_type not in ["distilkobert", "xlm-roberta"]:
                inputs["token_type_ids"] = batch[2]  # Distilkobert, XLM-Roberta don't use segment_ids
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs["labels"].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs["labels"].detach().cpu().numpy(), axis=0)

AttributeError: 'collections.OrderedDict' object has no attribute 'device'

In [197]:
args = torch.load(os.path.join("./model", 'koELECTRA_epoch7'))

In [198]:
args

OrderedDict([('electra.embeddings.word_embeddings.weight',
              tensor([[-0.0283,  0.0494, -0.0420,  ..., -0.0797,  0.0883, -0.0482],
                      [-0.0239, -0.0071,  0.1551,  ...,  0.0486, -0.1820, -0.0018],
                      [-0.0736,  0.0154, -0.0066,  ..., -0.1031,  1.0226,  0.0610],
                      ...,
                      [-0.0281,  0.0449, -0.0198,  ..., -0.0735,  0.0658, -0.0386],
                      [-0.0325,  0.0351, -0.0340,  ..., -0.0739,  0.0923, -0.0515],
                      [-0.0292,  0.0421, -0.0447,  ..., -0.0719,  0.0842, -0.0380]],
                     device='cuda:0')),
             ('electra.embeddings.position_embeddings.weight',
              tensor([[-5.7810e-02,  4.0580e-02,  3.1330e-03,  ..., -7.6071e-02,
                        1.2335e+00,  7.0402e-02],
                      [ 3.6186e-03,  7.6510e-03,  5.8032e-02,  ...,  2.4272e-02,
                       -1.1143e-01,  9.9718e-03],
                      [ 1.3220e-02, -3.6738e

In [200]:
model = AutoModelForSequenceClassification.from_pretrained('./ckpt/koelectra-base-v2-nsmc-ckpt/checkpoint-2000')  # Config will be automatically loaded from model_dir
model.to("cuda")
model.eval()
logger.info("***** Model Loaded *****")

In [201]:
model = load_model

NameError: name 'load_model' is not defined

In [202]:
result = []
for batch in dataloader:
    batch = tuple(t.to("cuda") for t in batch)
    with torch.no_grad():
        inputs = {"input_ids": batch[0],
              "attention_mask": batch[1],
              "labels": None}
        outputs = model(**inputs)
        logits = outputs[0]
        print(logits)
        result.append(logits)
print(len(result))

tensor([[ 8.1487e-02, -2.0092e-01],
        [-4.9352e-02, -1.2332e-01],
        [ 7.8471e-02, -2.3849e-01],
        [-3.9268e-02, -7.4065e-02],
        [ 2.5782e-02, -2.0974e-01],
        [ 4.3766e-03, -1.3649e-01],
        [-1.2409e-03, -1.5019e-01],
        [ 9.6684e-02, -2.1868e-01],
        [ 1.0886e-01, -2.4166e-01],
        [ 1.2199e-01, -2.2383e-01],
        [ 1.3190e-02, -1.2043e-01],
        [ 8.5075e-02, -2.5102e-01],
        [ 1.3123e-01, -2.3997e-01],
        [-1.1926e-04, -1.4805e-01],
        [ 7.7764e-02, -2.1178e-01],
        [ 5.9854e-02, -1.6608e-01],
        [ 7.8130e-02, -1.8938e-01],
        [ 1.2916e-01, -2.3374e-01],
        [ 1.0572e-01, -2.2612e-01],
        [ 4.4560e-02, -1.6021e-01],
        [ 3.7387e-02, -1.4721e-01],
        [ 1.8608e-02, -1.4677e-01],
        [ 2.0427e-02, -1.5380e-01],
        [ 1.1802e-01, -2.1344e-01],
        [ 5.1566e-03, -2.0435e-01],
        [ 7.1172e-02, -2.4105e-01],
        [ 1.1058e-01, -2.0622e-01],
        [ 1.2500e-01, -2.609

tensor([[ 0.0425, -0.2293],
        [ 0.1381, -0.2520],
        [-0.0288, -0.1040],
        [ 0.1026, -0.2650],
        [ 0.0841, -0.1881],
        [ 0.0612, -0.1692],
        [ 0.0239, -0.2202],
        [ 0.0969, -0.2407],
        [ 0.1115, -0.2267],
        [ 0.1332, -0.2145],
        [ 0.0611, -0.2133],
        [ 0.0983, -0.2879],
        [ 0.0908, -0.2000],
        [ 0.1210, -0.2235],
        [ 0.0994, -0.2132],
        [-0.0064, -0.1935],
        [ 0.1306, -0.2145],
        [ 0.0607, -0.1676],
        [ 0.0653, -0.1692],
        [ 0.0297, -0.1394],
        [ 0.1246, -0.2468],
        [ 0.1217, -0.2388],
        [ 0.0624, -0.2128],
        [ 0.0008, -0.1815],
        [-0.0287, -0.1632],
        [ 0.0964, -0.2047],
        [ 0.0190, -0.1981],
        [ 0.1122, -0.2096],
        [ 0.0667, -0.2668],
        [ 0.1308, -0.2420],
        [ 0.0161, -0.1937],
        [-0.0192, -0.1899],
        [ 0.0141, -0.1986],
        [ 0.0692, -0.2655],
        [ 0.0809, -0.2550],
        [ 0.1011, -0

tensor([[ 0.0425, -0.1549],
        [ 0.1009, -0.2632],
        [ 0.1356, -0.2377],
        [ 0.0606, -0.2304],
        [-0.0128, -0.1155],
        [ 0.0552, -0.1903],
        [ 0.0448, -0.1525],
        [ 0.0926, -0.2714],
        [ 0.1345, -0.2201],
        [ 0.0693, -0.2145],
        [ 0.0577, -0.2288],
        [ 0.0729, -0.2031],
        [ 0.1216, -0.2413],
        [-0.0375, -0.0984],
        [ 0.1246, -0.2333],
        [ 0.0574, -0.2139],
        [ 0.0142, -0.1458],
        [-0.0218, -0.1369],
        [ 0.0443, -0.1694],
        [-0.0087, -0.2378],
        [ 0.0678, -0.2270],
        [ 0.1415, -0.2499],
        [ 0.0882, -0.2077],
        [ 0.0569, -0.1847],
        [ 0.0325, -0.2108],
        [ 0.1450, -0.2411],
        [ 0.0016, -0.1267],
        [ 0.0025, -0.1341],
        [ 0.1205, -0.2597],
        [ 0.1195, -0.2418],
        [ 0.0513, -0.2479],
        [ 0.0714, -0.1749],
        [ 0.0171, -0.2102],
        [-0.0015, -0.1204],
        [ 0.1191, -0.2257],
        [ 0.0681, -0

tensor([[ 0.0570, -0.1677],
        [ 0.0893, -0.2333],
        [ 0.1062, -0.1956],
        [ 0.0071, -0.1972],
        [-0.0141, -0.1137],
        [-0.0310, -0.2307],
        [-0.0074, -0.1660],
        [ 0.0417, -0.1739],
        [ 0.1004, -0.2449],
        [ 0.0817, -0.2510],
        [ 0.0946, -0.2228],
        [ 0.1263, -0.2183],
        [ 0.1252, -0.2233],
        [ 0.1109, -0.2345],
        [ 0.0801, -0.2173],
        [ 0.1189, -0.2659],
        [ 0.1119, -0.2393],
        [ 0.0827, -0.1923],
        [ 0.0525, -0.2496],
        [ 0.1432, -0.2712],
        [ 0.1435, -0.2241],
        [ 0.1240, -0.2308],
        [ 0.0239, -0.1581],
        [ 0.0973, -0.1952],
        [ 0.1326, -0.2352],
        [ 0.1378, -0.2313],
        [ 0.1284, -0.2225],
        [ 0.1136, -0.2553],
        [ 0.1054, -0.2350],
        [ 0.1198, -0.1929],
        [ 0.0754, -0.1746],
        [ 0.0568, -0.2187],
        [ 0.0623, -0.1839],
        [ 0.0403, -0.1547],
        [ 0.1177, -0.2388],
        [ 0.0918, -0

tensor([[ 0.1141, -0.2210],
        [ 0.0346, -0.1787],
        [ 0.1096, -0.2362],
        [ 0.0752, -0.1761],
        [ 0.1296, -0.2245],
        [ 0.1184, -0.2466],
        [ 0.1035, -0.2592],
        [ 0.0189, -0.1812],
        [ 0.1098, -0.2695],
        [-0.0478, -0.1462],
        [ 0.1324, -0.2348],
        [ 0.1132, -0.2608],
        [ 0.1119, -0.2399],
        [ 0.0354, -0.2409],
        [-0.0046, -0.1989],
        [ 0.1219, -0.2140],
        [ 0.1422, -0.2219],
        [ 0.0101, -0.1647],
        [ 0.1161, -0.2062],
        [ 0.0796, -0.2101],
        [ 0.0918, -0.2369],
        [ 0.0684, -0.2291],
        [ 0.1162, -0.2308],
        [ 0.0962, -0.2161],
        [ 0.1195, -0.2422],
        [ 0.1203, -0.2295],
        [ 0.1220, -0.2150],
        [ 0.1129, -0.2161],
        [ 0.0359, -0.2157],
        [ 0.1098, -0.2171],
        [ 0.0461, -0.1526],
        [ 0.1348, -0.2323],
        [ 0.0279, -0.2520],
        [ 0.0078, -0.1646],
        [ 0.1183, -0.2255],
        [ 0.1300, -0

...

tensor([[ 0.0496, -0.1572],
        [ 0.0775, -0.2240],
        [ 0.0037, -0.2282],
        [-0.0445, -0.1968],
        [ 0.0551, -0.2244],
        [-0.0287, -0.0803],
        [ 0.0298, -0.1885],
        [ 0.0565, -0.1940],
        [ 0.1144, -0.2170],
        [ 0.1159, -0.2182],
        [ 0.1170, -0.2570],
        [ 0.1207, -0.2327],
        [ 0.0549, -0.2009],
        [ 0.1232, -0.2642],
        [ 0.0014, -0.1769],
        [ 0.0576, -0.1645],
        [ 0.0552, -0.1812],
        [ 0.1193, -0.2404],
        [ 0.1215, -0.2409],
        [ 0.0658, -0.2086],
        [ 0.1291, -0.2396],
        [ 0.0597, -0.1879],
        [ 0.1256, -0.2217],
        [-0.0219, -0.1400],
        [ 0.1154, -0.2387],
        [ 0.0274, -0.2231],
        [ 0.0665, -0.2275],
        [ 0.0912, -0.2287],
        [ 0.0164, -0.2238],
        [ 0.0255, -0.1580],
        [-0.0065, -0.1243],
        [-0.0083, -0.1696],
        [ 0.1090, -0.2503],
        [ 0.0504, -0.1802],
        [ 0.1101, -0.2303],
        [ 0.0928, -0

tensor([[ 5.3045e-02, -2.0587e-01],
        [-2.6423e-02, -1.6655e-01],
        [ 9.0701e-02, -2.3596e-01],
        [ 4.5777e-02, -1.6679e-01],
        [ 1.9703e-02, -1.9700e-01],
        [-7.6273e-02, -3.9798e-02],
        [-4.6343e-04, -1.6790e-01],
        [ 3.5343e-02, -1.5251e-01],
        [ 1.2349e-01, -2.4625e-01],
        [ 5.4452e-02, -2.0674e-01],
        [ 6.3008e-02, -1.8689e-01],
        [ 1.0039e-01, -2.5365e-01],
        [ 1.0116e-01, -2.4849e-01],
        [ 1.8355e-02, -1.7955e-01],
        [ 9.7360e-02, -2.0921e-01],
        [-2.6336e-02, -1.8279e-01],
        [ 3.1878e-02, -1.7134e-01],
        [ 8.5459e-02, -1.9761e-01],
        [ 1.1013e-01, -2.2981e-01],
        [ 1.1939e-01, -2.8570e-01],
        [ 1.0559e-01, -2.4155e-01],
        [ 1.2570e-01, -2.3450e-01],
        [ 6.3606e-02, -2.1875e-01],
        [-2.7821e-02, -1.2151e-01],
        [-5.1208e-02, -1.1845e-01],
        [ 1.1892e-01, -2.2493e-01],
        [ 9.8933e-02, -1.9748e-01],
        [ 1.4068e-01, -2.272

tensor([[ 0.1005, -0.2337],
        [ 0.1000, -0.2454],
        [ 0.1024, -0.2584],
        [ 0.1001, -0.2378],
        [ 0.0966, -0.2496],
        [ 0.1152, -0.2206],
        [ 0.0386, -0.1909],
        [ 0.0078, -0.1924],
        [ 0.0538, -0.2309],
        [ 0.0643, -0.2388],
        [ 0.0722, -0.1841],
        [-0.0461, -0.1564],
        [ 0.1198, -0.2477],
        [-0.0160, -0.2166],
        [ 0.1385, -0.2433],
        [ 0.1387, -0.2173],
        [-0.0315, -0.1639],
        [ 0.0249, -0.1887],
        [ 0.0352, -0.1769],
        [ 0.1124, -0.2462],
        [ 0.0253, -0.1457],
        [ 0.0406, -0.1796],
        [ 0.1083, -0.2213],
        [ 0.1087, -0.2200],
        [ 0.0732, -0.2290],
        [ 0.0325, -0.1696],
        [ 0.1121, -0.2000],
        [ 0.1203, -0.2324],
        [-0.0418, -0.1208],
        [ 0.0381, -0.1717],
        [ 0.0373, -0.2030],
        [ 0.0673, -0.2476],
        [ 0.1235, -0.2056],
        [-0.0375, -0.1744],
        [ 0.1075, -0.2548],
        [ 0.1137, -0

tensor([[ 0.0036, -0.1712],
        [ 0.0236, -0.1621],
        [ 0.1433, -0.2263],
        [ 0.0336, -0.1477],
        [-0.0609, -0.1546],
        [ 0.0876, -0.2469],
        [ 0.1115, -0.2200],
        [ 0.0570, -0.2191],
        [ 0.1200, -0.2408],
        [-0.0402, -0.2063],
        [ 0.0529, -0.1758],
        [ 0.1084, -0.2338],
        [ 0.1317, -0.2493],
        [-0.0080, -0.1933],
        [-0.0274, -0.1744],
        [ 0.0851, -0.2395],
        [ 0.1136, -0.2267],
        [ 0.0387, -0.1701],
        [ 0.0107, -0.1151],
        [ 0.0547, -0.1694],
        [ 0.0644, -0.2056],
        [ 0.1335, -0.2734],
        [ 0.0157, -0.2022],
        [ 0.0118, -0.2001],
        [ 0.1146, -0.2243],
        [ 0.0887, -0.2280],
        [ 0.0170, -0.1361],
        [-0.0112, -0.1575],
        [ 0.0183, -0.1327],
        [ 0.1210, -0.2545],
        [ 0.1145, -0.2326],
        [-0.0142, -0.2491],
        [ 0.0743, -0.2064],
        [ 0.0856, -0.2443],
        [ 0.1252, -0.2284],
        [ 0.1267, -0

tensor([[ 0.1032, -0.2672],
        [ 0.1207, -0.2396],
        [ 0.1149, -0.2543],
        [ 0.1213, -0.2758],
        [ 0.0271, -0.1882],
        [ 0.0793, -0.2517],
        [ 0.0781, -0.2351],
        [ 0.0116, -0.1296],
        [ 0.0251, -0.1332],
        [ 0.0625, -0.1949],
        [ 0.0723, -0.1853],
        [ 0.1295, -0.2588],
        [-0.0042, -0.1142],
        [ 0.1228, -0.2510],
        [ 0.1316, -0.2432],
        [ 0.0829, -0.2499],
        [-0.0506, -0.2054],
        [-0.0254, -0.1298],
        [-0.0355, -0.1556],
        [ 0.0589, -0.1871],
        [ 0.0915, -0.2216],
        [ 0.1006, -0.2590],
        [ 0.0796, -0.2114],
        [ 0.0555, -0.1635],
        [-0.0322, -0.0658],
        [ 0.0705, -0.2297],
        [-0.0124, -0.1116],
        [ 0.0646, -0.2398],
        [ 0.1270, -0.2171],
        [-0.0135, -0.0980],
        [ 0.1119, -0.2221],
        [ 0.1271, -0.2317],
        [ 0.0385, -0.1565],
        [ 0.1248, -0.2749],
        [-0.0051, -0.1190],
        [ 0.0298, -0

In [203]:
result_val = []
for batch_result in result:
    a = batch_result.tolist()
    for i in a:
        print(i)
        if i[0] > i[1]:
            result_val.append(0)
        else:
            result_val.append(1)

[0.08148691803216934, -0.2009236216545105]
[-0.04935155063867569, -0.12332257628440857]
[0.07847094535827637, -0.2384912073612213]
[-0.03926782310009003, -0.07406534254550934]
[0.025782085955142975, -0.20973576605319977]
[0.0043766312301158905, -0.1364857405424118]
[-0.0012408755719661713, -0.15018925070762634]
[0.09668417274951935, -0.21867690980434418]
[0.1088571771979332, -0.24166065454483032]
[0.12198606133460999, -0.22383081912994385]
[0.01319025456905365, -0.12043024599552155]
[0.08507494628429413, -0.2510247528553009]
[0.13122758269309998, -0.23996509611606598]
[-0.00011926144361495972, -0.14804567396640778]
[0.077763631939888, -0.21177998185157776]
[0.05985409393906593, -0.16607743501663208]
[0.07813044637441635, -0.18938079476356506]
[0.12915679812431335, -0.23374122381210327]
[0.10571527481079102, -0.22612042725086212]
[0.04455983266234398, -0.16021372377872467]
[0.03738655894994736, -0.14720644056797028]
[0.01860792189836502, -0.14676858484745026]
[0.020427117124199867, -0.1

[0.0026461277157068253, -0.1816181093454361]
[0.04647276923060417, -0.1601385474205017]
[0.07112562656402588, -0.2570401430130005]
[0.10404114425182343, -0.29136553406715393]
[0.09269849210977554, -0.21370667219161987]
[0.14677533507347107, -0.2408505380153656]
[0.11487741023302078, -0.2638745903968811]
[-0.010940492153167725, -0.09657277166843414]
[0.09622427821159363, -0.20031674206256866]
[0.10978434234857559, -0.22823509573936462]
[0.1214456632733345, -0.2380927950143814]
[0.09347977489233017, -0.22293055057525635]
[0.059009019285440445, -0.2443283498287201]
[0.08119232207536697, -0.196904718875885]
[0.10867146402597427, -0.22352807223796844]
[0.06688733398914337, -0.21710684895515442]
[0.1246819943189621, -0.23758400976657867]
[0.021748241037130356, -0.15290586650371552]
[0.07472780346870422, -0.1649017035961151]
[0.05061574652791023, -0.20534153282642365]
[-0.020173758268356323, -0.13881734013557434]
[0.15080441534519196, -0.25130099058151245]
[0.019256241619586945, -0.1555632054

[-0.03388863429427147, -0.09328457713127136]
[0.13060978055000305, -0.25680607557296753]
[0.1243862509727478, -0.25131359696388245]
[0.07530620694160461, -0.23155102133750916]
[0.12793171405792236, -0.22356699407100677]
[-0.04016992449760437, -0.13239213824272156]
[0.021981023252010345, -0.17944040894508362]
[0.10981354862451553, -0.23631471395492554]
[0.041592787951231, -0.17663723230361938]
[0.12334349006414413, -0.25918880105018616]
[-0.06078450754284859, -0.06169106811285019]
[0.10132691264152527, -0.23781755566596985]
[0.05135355889797211, -0.15586857497692108]
[0.03867293894290924, -0.17960074543952942]
[0.06616193801164627, -0.2372085303068161]
[0.1011003628373146, -0.20840679109096527]
[0.07957999408245087, -0.18988680839538574]
[0.06808814406394958, -0.19877779483795166]
[0.07000593841075897, -0.24755986034870148]
[0.12182745337486267, -0.2321009486913681]
[-0.009596854448318481, -0.17737635970115662]
[0.12665066123008728, -0.2254796028137207]
[0.07058264315128326, -0.17788513

[-0.028697146102786064, -0.15341588854789734]
[0.012725137174129486, -0.19681893289089203]
[-0.00079304538667202, -0.16064327955245972]
[0.11159224063158035, -0.220978245139122]
[-0.040604185312986374, -0.1296066790819168]
[0.12366774678230286, -0.2315087914466858]
[-0.04121585190296173, -0.1762448102235794]
[0.10741978883743286, -0.2812610864639282]
[0.10147122293710709, -0.24903222918510437]
[0.12343905121088028, -0.23178371787071228]
[0.07296494394540787, -0.2698037922382355]
[0.11032522469758987, -0.22594937682151794]
[0.0915011540055275, -0.20155496895313263]
[0.11811687797307968, -0.2588472366333008]
[0.07270624488592148, -0.2366989552974701]
[-0.001795932650566101, -0.16289596259593964]
[0.1254958063364029, -0.24811658263206482]
[0.09129858016967773, -0.18645891547203064]
[-0.025544892996549606, -0.13178196549415588]
[-0.016952404752373695, -0.1876838207244873]
[0.030514411628246307, -0.24488280713558197]
[0.06853468716144562, -0.16873079538345337]
[0.07516704499721527, -0.17611

[0.016256164759397507, -0.12740038335323334]
[0.10478077083826065, -0.1871664673089981]
[0.08237270265817642, -0.2240820825099945]
[0.00015833228826522827, -0.13275599479675293]
[0.11410659551620483, -0.20314040780067444]
[0.10482810437679291, -0.2343229502439499]
[-0.0019427761435508728, -0.1182054728269577]
[0.07659599930047989, -0.24775347113609314]
[0.020710542798042297, -0.21706511080265045]
[0.12175876647233963, -0.21653497219085693]
[-0.03367258608341217, -0.1302804946899414]
[0.1429048478603363, -0.23384693264961243]
[0.1245676577091217, -0.27467915415763855]
[0.025405384600162506, -0.1635097712278366]
[0.041463255882263184, -0.2488766312599182]
[0.12748979032039642, -0.2710556983947754]
[0.13523012399673462, -0.2357337921857834]
[0.0943339541554451, -0.22115997970104218]
[0.06346730887889862, -0.1968427151441574]
[0.1190485805273056, -0.24813511967658997]
[0.08762411773204803, -0.24026142060756683]
[0.07927146553993225, -0.20881132781505585]
[0.056343384087085724, -0.224669694

[0.11775291711091995, -0.2315416932106018]
[0.01021786779165268, -0.14360906183719635]
[0.08050835132598877, -0.22939814627170563]
[0.09945410490036011, -0.2586943209171295]
[0.09916704148054123, -0.24787752330303192]
[0.052239228039979935, -0.22730891406536102]
[0.12262419611215591, -0.23081916570663452]
[0.06978468596935272, -0.1974765509366989]
[-0.012164058163762093, -0.13848166167736053]
[0.017627619206905365, -0.20056931674480438]
[0.07804165780544281, -0.191632941365242]
[0.03959920257329941, -0.16242162883281708]
[0.1215958371758461, -0.25362229347229004]
[0.12042399495840073, -0.21697793900966644]
[0.014320813119411469, -0.145636647939682]
[-0.002764136530458927, -0.14338847994804382]
[0.01967903971672058, -0.14846815168857574]
[0.11698359251022339, -0.25578272342681885]
[-0.0072082579135894775, -0.18205422163009644]
[0.11420433223247528, -0.2666935920715332]
[0.07956552505493164, -0.24231654405593872]
[0.11512633413076401, -0.24229948222637177]
[0.03967971354722977, -0.218079

[0.13581444323062897, -0.23429054021835327]
[0.10150708258152008, -0.23142072558403015]
[0.03583816438913345, -0.16611617803573608]
[0.09489069879055023, -0.2276725172996521]
[0.1025751382112503, -0.24770128726959229]
[-0.083767369389534, -0.14348861575126648]
[0.09372515976428986, -0.25606635212898254]
[0.060447923839092255, -0.24831663072109222]
[0.06455857306718826, -0.2215314507484436]
[0.09493401646614075, -0.23744316399097443]
[0.02439330331981182, -0.21307887136936188]
[0.08946395665407181, -0.18833278119564056]
[-0.022325480356812477, -0.18821240961551666]
[0.09225127846002579, -0.25909024477005005]
[0.029825540259480476, -0.17504838109016418]
[0.11061412841081619, -0.20298248529434204]
[-0.012251762673258781, -0.11621508747339249]
[0.11377871036529541, -0.24077533185482025]
[0.0023070769384503365, -0.19416126608848572]
[0.023693392053246498, -0.13800185918807983]
[0.0016627795994281769, -0.11081486940383911]
[0.07042699307203293, -0.18984940648078918]
[0.033794280141592026, -0

[0.07812834531068802, -0.2619503438472748]
[-0.034267526119947433, -0.06568868458271027]
[0.10285686701536179, -0.23296386003494263]
[0.11353851109743118, -0.2420850396156311]
[-0.021577831357717514, -0.18660655617713928]
[0.015796352177858353, -0.13150757551193237]
[0.006571488454937935, -0.2133980393409729]
[0.015042588114738464, -0.2069929540157318]
[0.011175274848937988, -0.16982217133045197]
[0.10878397524356842, -0.2333734631538391]
[0.14458873867988586, -0.23311209678649902]
[0.002967621199786663, -0.2069033682346344]
[0.0941888615489006, -0.23111747205257416]
[0.09478812664747238, -0.22613480687141418]
[0.09665098786354065, -0.21074116230010986]
[-0.03138299286365509, -0.1542171835899353]
[0.10852960497140884, -0.2315659373998642]
[0.14625585079193115, -0.2440563589334488]
[0.0506618469953537, -0.19107499718666077]
[0.10242407768964767, -0.20681102573871613]
[0.03571175038814545, -0.1865302473306656]
[-0.02528535947203636, -0.15162412822246552]
[0.12548215687274933, -0.23234181

[0.12565933167934418, -0.24191373586654663]
[0.12182355672121048, -0.23064649105072021]
[0.14582008123397827, -0.24379760026931763]
[-0.05225115269422531, -0.13991282880306244]
[0.12140972912311554, -0.2702837586402893]
[0.010919880121946335, -0.16283473372459412]
[0.03245086967945099, -0.14405544102191925]
[0.13127733767032623, -0.22558483481407166]
[0.13426746428012848, -0.2425985485315323]
[-0.0807933434844017, -0.11319790780544281]
[0.13597707450389862, -0.2594720721244812]
[0.1050695925951004, -0.2497827112674713]
[0.11056005209684372, -0.2439468502998352]
[0.014131840318441391, -0.14272788166999817]
[0.08092524856328964, -0.19620878994464874]
[0.119764544069767, -0.23277242481708527]
[0.005860453471541405, -0.20591866970062256]
[0.11640363186597824, -0.21650348603725433]
[0.026074565947055817, -0.15589749813079834]
[0.12516199052333832, -0.2226201444864273]
[0.03707435727119446, -0.14944376051425934]
[0.055223409086465836, -0.24800139665603638]
[0.1300380527973175, -0.23786170780

[0.12248428910970688, -0.2373010367155075]
[0.1268431544303894, -0.20265798270702362]
[0.13043370842933655, -0.2317572832107544]
[-0.06338739395141602, -0.2091543972492218]
[0.04311450570821762, -0.19157908856868744]
[0.02046293020248413, -0.14088524878025055]
[0.09679365158081055, -0.23058122396469116]
[0.03788413107395172, -0.16629649698734283]
[0.020921476185321808, -0.23549240827560425]
[0.10184744000434875, -0.25254490971565247]
[-0.007386980578303337, -0.14578048884868622]
[0.060305994004011154, -0.198196142911911]
[0.012277995236217976, -0.25248512625694275]
[0.095791295170784, -0.24694721400737762]
[0.10924391448497772, -0.2464752346277237]
[0.10176383703947067, -0.2558426558971405]
[0.1348358541727066, -0.22589069604873657]
[0.0034409761428833008, -0.1610361933708191]
[0.06879059225320816, -0.2216356247663498]
[0.028825335204601288, -0.20803989470005035]
[0.13618159294128418, -0.24540631473064423]
[-0.02138938568532467, -0.08797310292720795]
[-0.0042043449357151985, -0.1707399

In [205]:
len(result_val)

11187

In [206]:
import csv

with open('laast.csv', 'w') as f:
    wr = csv.writer(f)
    wr.writerow(['Id', 'Predicted'])
    for i in range(len(result_val)):
        wr.writerow([i, result_val[i]])